# Import libraries

In [1]:
from bs4 import BeautifulSoup
import time
from datetime import datetime 
import json
import requests
import pandas as pd

In [2]:
def log_progress(sequence, name, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [3]:
# do validation and checks before insert
def validate_string(val):
   if val != None:
        if type(val) is int:
            #for x in val:
            #   print(x)
            return str(val)
        else:
            return val

In [4]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    if type(text) != int:
        text = text.replace('"','')
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)
    else:
        return text

## Prepare scrapper

First, lets get all Publishers and their numbers for later use.

In [5]:
url = "https://boardgamegeek.com/browse/boardgamepublisher/page/1050"

In [ ]:
%timeit
publishers = {}
for i in log_progress(range(1,2000),'Page'):
    url = "https://boardgamegeek.com/browse/boardgamepublisher/page/%d" %(i)
    list_page = BeautifulSoup(
    requests.get(url).text,
    "html.parser")
    for link in list_page.find_all('a'):
        publisher = str(link.get('href'))
        if link.get('href')[1:20] == 'boardgamepublisher/':
            publisher = publisher[20:]
            publisher = publisher.split('/')
            publishers[publisher[1]] = publisher[0]

In [36]:
print(len(publishers))

19248


## Get information about each Publisher

In [ ]:
publisher_games = {}
for publisher, publisher_id in log_progress(publishers, 'Publisher'):
    url = """https://boardgamegeek.com/geekitem.php?instanceid=8&objecttype=company&
    objectid=%d
    &subtype=boardgamepublisher&pageid=1&sort=name&view=boardgames&modulename=linkeditems&
    callback=&showcount=10000&filters[categoryfilter]=&filters[mechanicfilter]=&action=linkeditems&ajax=1""" % (publisher_id)
    
    list_page = BeautifulSoup(
        requests.get(url).text,
        "html.parser")
    # Loop through the list of games, finding each of the URLs in the table
    
    all_games = []
    number_of_items = len(list_page.find_all("span", {"class": "go_thumbnail"}))
    i = 0

    all_pages = list_page.find_all("span", {"class": "go_thumbnail"})
    for game in log_progress(all_pages, 'Games'):

        game_url = "https://boardgamegeek.com" + game.find("a").get("href")
         # Download the game page and load it into memory
        game_page = BeautifulSoup(
             requests.get(game_url).text,
             "html.parser"
         )

        script = game_page.find('script').get_text()
        json_text = script[script.find('GEEK.geekitemPreload')+23:script.find('GEEK.geekitemSettings')-3]
        json_raw_data = json.loads(json_text)
        all_games.append(json_raw_data)
        time.sleep(0.5)
    publisher_games[publisher] = all_games
    #     i += 1
    #     if i == 4:
    #         break

## Importing JSON to MySql

In [26]:
import pymysql, os
from datetime import date
import calendar

In [27]:
# connect to MySQL
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Esqueci1',
                             db='bgg_information',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

## Insert Items

#### Get last ID on Stats table

In [28]:
sql = 'Select MAX(bgg_stats) from game_stats GROUP BY bgg_stats;'
last_record = cursor.execute(sql)

#### Fields on the table

In [29]:
what_to_insert = ['subtypename','itemid','objecttype', 'objectid', 'label', 'name', 'yearpublished', 'minplayers',
 'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'description','date_downloaded', 'bgg_stats']

#### Get current time

In [30]:
date_downloaded = calendar.timegm(time.gmtime())

#### Transformation from JSON to Database

First, we create the table for games general information.
Since the other tables have auto increment Primary keys, we can get the max value, add one and start sdding here before creating records.

In [31]:
sql = 'INSERT INTO bgg_information ('
for item in what_to_insert:
    sql += '%s, ' % (item)
sql = sql[:-2] + ') VALUES ('
for i in range(0, len(all_games)):
    game = all_games[i]
    game_item = game['item']
    # parse json data to SQL insert
    for item in what_to_insert:
        if item == 'date_downloaded':
            sql += '"%s", ' % (date_downloaded)
        elif item == 'bgg_stats':
            last_record += 1
            sql += '"%s", ' % (last_record)
        else:
            sql += '"%s", ' % (remove_html_tags(game_item[item]))
            
    sql = sql[:-2] + '), ('
sql = sql[:-3] + ';'

In [32]:
cursor.execute(sql)
connection.commit()

## Insert Stats

In [33]:
what_to_insert = ['usersrated', 'average', 'baverage', 'stddev', 'avgweight', 'numweights', 'numgeeklists',
 'numtrading', 'numwanting', 'numwish', 'numowned', 'numprevowned', 'numcomments', 'numwishlistcomments',
 'numhasparts', 'numwantparts', 'views', 'playmonth', 'numplays','numplays_month', 'numfans']

In [34]:
date_downloaded = date.today().strftime('%Y-%m-%d')

sql = 'INSERT INTO game_stats ('
for item in what_to_insert:
    sql += '%s, ' % (item)
sql = sql[:-2] + ') VALUES ('
for i in range(0, len(all_games)):
    game_stats = all_games[i]['item']['stats']
    # parse json data to SQL insert
    for item in what_to_insert:
        sql += '"%s", ' % (remove_html_tags(game_stats[item]))
    sql = sql[:-2] + '), ('
    
    all_games[i]['item']
sql = sql[:-3] + ';'

In [35]:
cursor.execute(sql)
connection.commit()

In [36]:
connection.close()